In [1]:
import asyncio, time, sys
import websockets, json
import numpy as np
import syncmetrics as syncm
from numpy_ringbuffer import RingBuffer

from pythonosc import osc_message_builder
from pythonosc import osc_bundle_builder

/Users/williamprimett1/anaconda/lib/python3.5/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
#OSC input config   
ip = 'localhost'
port = 8888

#    Send to Processing/MAX/Etc...
def send_osc_bundle(dev_id, *values):
    bundle = osc_bundle_builder.OscBundleBuilder(osc_bundle_builder.IMMEDIATELY)
    msg = osc_message_builder.OscMessageBuilder(address=output_address)
    # Test bundle outputs: 1, 2, 3, ...
    for val in values:
        print (val)
        msg.add_arg(val)
        bundle.add_content(msg.build())
        bundle = bundle.build()
    client.send(bundle)
    
def output_individual(data, labels):
    data = json.loads(data)
    printJSON(data)
    bundle = osc_bundle_builder.OscBundleBuilder(
        osc_bundle_builder.IMMEDIATELY)

    for label in labels:
        output_address += self.device_number + "/bitalino/" + label
        msg = osc_message_builder.OscMessageBuilder(
            address=output_address)
        msg.add_arg(arg_to_add)
        msg.build()
        client.send(msg)

In [5]:
class Session:
    device_data = json.dumps({})
    uri = 'ws://localhost:8000'
    ws_timeout = 5.0
    num_users = 2
    
    # to do processing in windoes 
    fs=1000
    buffer_W=150
    win_size=15*fs
    window_A = np.zeros(win_size)
    window_B = np.zeros(win_size)
    no_buffer=int(win_size/buffer_W)
    buffers = RingBuffer(capacity=no_buffer, dtype=(float, num_users))

    def calculate_features(self):
        device_data = json.loads(self.device_data)
        self.buffers.appendleft([device_data["A1"][0], device_data["A2"][0]])
        
        if self.buffers.shape[0] > self.no_buffer-1:
            buff_A = np.array([i[0] for i in np.array(self.buffers)])
            buff_B = np.array([i[1] for i in np.array(self.buffers)])
            result_1=syncm.lin_reg_r_metric(buff_A, buff_B)
            result_2=syncm.cos_similarity(buff_A, buff_B).flat[0]

            #add results here
            results = [result_1, result_2]
            print(results)
#             send_osc_bundle(result_1)

    async def ServerBIT_recieve(self, uri):
        async with websockets.connect(uri) as websocket:
            while True:
                self.device_data = await websocket.recv()
                self.calculate_features()
                await asyncio.sleep(0.0)
                
    def run(self):
        uri = self.uri
        print("Serving on {}".format(uri))
        try:
            asyncio.get_event_loop().run_until_complete(
                self.ServerBIT_recieve(uri))
        except Exception as e:
            print(e)
            pass
        finally:
            print("no connection, try again")
            exit

In [ ]:
session = Session()
session.run()

Serving on ws://localhost:8000
[0.39674763599691343, array([[0.99999562]])]
[0.3672895992284589, array([[0.9999954]])]
[0.30437962739571783, array([[0.99999514]])]
[0.2536925132202285, array([[0.99999501]])]
[0.23809672753028846, array([[0.99999488]])]
[0.21561219226877396, array([[0.99999467]])]
[0.21390398951098316, array([[0.99999467]])]
[0.1840458715758926, array([[0.99999437]])]
[0.12222969154802872, array([[0.99999379]])]
[0.07193945265899203, array([[0.99999337]])]
[0.02426757739686504, array([[0.99999295]])]
[-0.02914027700085301, array([[0.99999234]])]
[-0.08446579726276496, array([[0.99999179]])]
[-0.1360268326666796, array([[0.99999125]])]
[-0.17689707665862103, array([[0.99999059]])]
[-0.2276283473627505, array([[0.99998987]])]
[-0.27434098473508484, array([[0.99998918]])]
[-0.3272622148532775, array([[0.99998861]])]
[-0.3810881994599607, array([[0.99998781]])]
[-0.4313875279918303, array([[0.9999869]])]
[-0.503693030760406, array([[0.99998481]])]
[-0.5628302185553073, arra